In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression

## Load Iris dataset

Load it and create three One vs. All solutions to combine and determine the class of each entry. The $y$ and $W$ are created to make the fitting easier through looping.

In [2]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
X = np.c_[np.ones((X.shape[0], 1)), X]
# Convert to three One vs. All problems
y = np.vstack([np.array(y == 0, dtype=int),
               np.array(y == 1, dtype=int),
               np.array(y == 2, dtype=int),
               ]).reshape(3, -1, 1)
W = np.random.randn(3, X.shape[1], 1)

In [3]:
W.reshape(3, -1)

array([[-0.35046944,  1.3034533 , -0.3741031 ,  0.29645638,  0.10589084],
       [-0.0358767 , -0.45248151, -0.50465058, -0.15710759,  0.37792978],
       [ 0.45238857,  0.29891665,  1.55690124, -0.15699958,  1.05677563]])

### Define some helper functions to make life easier.

In [4]:
def sigmoid(z):
    """Returns sigmoid of array z"""
    return 1. / (1 + np.exp(-z))

def predict(X, W):
    """Predicts classes from data"""
    pred = sigmoid(np.dot(X, W).reshape(X.shape[0], 3))
    return pred.argmax(axis=1)

def score(y_pred, y):
    return (y_pred == y).mean()

### Grad. Descent

In [5]:
n_iters = 100000
alpha = 0.1
for i, w in enumerate(W):
    for _ in range(n_iters):
        w -= (alpha/X.shape[0]) * np.dot(X.T, sigmoid(np.dot(X, w)) - y[i])

np.round(W.reshape(3, -1), 4)

array([[  0.22  ,   1.5966,   3.3937,  -6.3151,  -2.9521],
       [  7.3783,  -0.2453,  -2.7965,   1.3136,  -2.7783],
       [-15.086 ,  -3.7178,  -5.219 ,   6.9741,  11.2953]])

## Compare to scikit-learn

In [6]:
logReg = LogisticRegression(solver='lbfgs', multi_class='ovr', fit_intercept=False, penalty='none')
_, y_data = load_iris(return_X_y=True)
logReg.fit(X, y_data)
np.round(logReg.coef_, 4)

array([[  1.1849,   2.0216,   6.9985, -11.1481,  -5.1549],
       [  7.3785,  -0.2454,  -2.7966,   1.3137,  -2.7784],
       [-42.6376,  -2.4652,  -6.6809,   9.4292,  18.2866]])

### Accuracy scores
I tried to make the two models as similar as possible. Weights look fairly similar (relative strengths and pos/neg within classes). I wouldn't expect them to be the same since they're using a different solver than my brute force approach. 

In [7]:
print("My score: ", score(predict(X, W), y_data))
print("Sklearn score: ", logReg.score(X, y_data))

My score:  0.98
Sklearn score:  0.98
